In [1]:
import astropy.units as un
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import Angle, SkyCoord
from astropy.cosmology import Planck18
from astropy.io import fits
from astropy.stats import median_absolute_deviation as mad
from astropy.units import Quantity
from astropy.wcs import WCS
from regions import CirclePixelRegion, CircleSkyRegion, PixCoord

In [2]:
redshift_z = 0.034
dist = Planck18.comoving_distance(redshift_z)
scale = Planck18.arcsec_per_kpc_comoving(redshift_z)

In [3]:
def get_nbeams(data, header):
    dx = -header["CDELT1"]
    dy = header["CDELT2"]
    bmaj = 12.381457328796387 * un.arcsec
    bmin = 11.684733390808105 * un.arcsec
    area_pixels = np.pi * bmaj.to(un.deg) * bmin.to(un.deg) / (4. * np.log(2)) / (dx * dy)
    nbeams = np.count_nonzero(~np.isnan(data)) / area_pixels.value
    return nbeams

In [4]:
def get_image_coordinates(header, w):
    m = header["NAXIS1"]
    n = header["NAXIS2"]
    try:
        frame = header["RADESYS"].lower()
    except KeyError:
        frame = "fk5"
    x = np.arange(0, m, 1)
    y = np.arange(0, n, 1)
    xx, yy = np.meshgrid(x, y)

    ra, dec = w.all_pix2world(xx, yy, 0) * un.deg
    sky_coord = SkyCoord(ra=ra, dec=dec, frame=frame)

    return sky_coord

In [5]:
def get_statistics(rm, sigma, header):
    avg_rm = np.nanmean(rm)
    med_rm = np.nanmedian(rm)
    mad_rm = np.nanmedian(np.abs(rm - np.nanmedian(rm)))
    sigma_rm_obs = np.nanstd(rm)
    med_sigma_phi = np.nanmedian(sigma)
    sigma_rm = np.sqrt(sigma_rm_obs**2 - med_sigma_phi**2)
    nbeams = get_nbeams(rm, header)

    print("<RM> : {0:.2f}".format(avg_rm))
    print("sigma(RM): {0:.2f}".format(sigma_rm))
    print("med(RM): {0:.2f}".format(med_rm))
    print("MAD(RM): {0:.2f}".format(mad_rm))
    print("med(sigma_phi): {0:.2f}".format(med_sigma_phi))
    print("sigma(RM)_obs: {0:.2f}".format(sigma_rm_obs))
    print("nbeams: {0:.2f}".format(nbeams))

In [6]:
def create_circular_skyregion(
    ra, dec, radius, cluster_center_coord, radius_unit="arcsec", unit="deg"
):
    center = SkyCoord(ra, dec, unit=unit)
    dist_to_center = center.separation(cluster_center_coord)
    dist_kpc = dist_to_center.arcsecond * un.arcsec / scale
    print(dist_kpc)
    radius = Angle(radius, radius_unit)
    region = CircleSkyRegion(center, radius)
    return region

In [7]:
cluster_center_coord = SkyCoord(ra=173.714 * un.deg, dec=49.091 * un.deg, frame="fk5")

In [8]:
rm_hdu = fits.open(
    "/home/miguel/Documents/datasets/A1314_JVLA/mfs/rm_images/delta_wavelet/max_faraday_depth.fits"
)[0]
p_hdu = fits.open(
    "/home/miguel/Documents/datasets/A1314_JVLA/mfs/rm_images/delta_wavelet/max_rotated_intensity.fits"
)[0]
sigma_qu_hdu = fits.open(
    "/home/miguel/Documents/datasets/A1314_JVLA/mfs/rm_images/delta_wavelet/sigma_qu_faraday.fits"
)[0]
sigma_phi_hdu = fits.open(
    "/home/miguel/Documents/datasets/A1314_JVLA/mfs/rm_images/delta_wavelet/sigma_phi_peak.fits"
)[0]
xray_hdu = fits.open("/home/miguel/Documents/datasets/A1314_JVLA/xray_interpolated.fits")[0]

In [9]:
# Core
# core_region = create_circular_skyregion(173.4962263, 49.0629333, 15.782, cluster_center_coord)
core_region = create_circular_skyregion(
    173.4962263, 49.0629333, 17.782, cluster_center_coord
)  # 5.01 nbeams

379.2163669411811 kpc


In [10]:
# North Lobe
north_region = create_circular_skyregion(173.4776803, 49.0761733, 41.295, cluster_center_coord)

405.5635803438024 kpc


In [11]:
# South Lobe
south_region = create_circular_skyregion(173.4719210, 49.0538024, 42.187, cluster_center_coord)

424.87499353091937 kpc


In [12]:
w = WCS(rm_hdu.header, naxis=2)
w_xray = WCS(xray_hdu.header, naxis=2)

Set OBSGEO-B to    34.078827 from OBSGEO-[XYZ].
Set OBSGEO-H to     2115.607 from OBSGEO-[XYZ]'. [astropy.wcs.wcs]
the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


In [13]:
sky_coord = get_image_coordinates(rm_hdu.header, w)

In [14]:
sky_coord_xray = get_image_coordinates(xray_hdu.header, w_xray)

In [15]:
core_mask = core_region.contains(sky_coord, w)
core_xray_mask = core_region.contains(sky_coord_xray, w_xray)
north_mask = north_region.contains(sky_coord, w)
north_xray_mask = north_region.contains(sky_coord_xray, w_xray)
south_mask = south_region.contains(sky_coord, w)
south_xray_mask = south_region.contains(sky_coord_xray, w_xray)

In [16]:
rm = rm_hdu.data
p = p_hdu.data
sigma_qu = sigma_qu_hdu.data
sigma_phi = sigma_phi_hdu.data
xray_data = xray_hdu.data

In [17]:
rm = np.where(p > 6 * sigma_qu, rm, np.nan)
sigma_phi = np.where(p > 6 * sigma_qu, sigma_phi, np.nan)

In [18]:
core_rm = rm[core_mask]
north_rm = rm[north_mask]
south_rm = rm[south_mask]

In [19]:
core_sigma = sigma_phi[core_mask]
north_sigma = sigma_phi[north_mask]
south_sigma = sigma_phi[south_mask]

In [20]:
core_xray = xray_data[core_xray_mask]
north_xray = xray_data[north_xray_mask]
south_xray = xray_data[south_xray_mask]

In [21]:
print("Core:")
get_statistics(core_rm, core_sigma, rm_hdu.header)
print("Mean xray: {0:.3f} c/s/deg2".format(np.nanmean(core_xray)))

Core:
<RM> : 103.78
sigma(RM): 50.86
med(RM): 136.50
MAD(RM): 6.82
med(sigma_phi): 0.44
sigma(RM)_obs: 50.86
nbeams: 5.01
Mean xray: 14.843 c/s/deg2


In [22]:
print("North lobe:")
get_statistics(north_rm, north_sigma, rm_hdu.header)
print("Mean xray: {0:.3f} c/s/deg2".format(np.nanmean(north_xray)))

North lobe:
<RM> : -11.27
sigma(RM): 30.33
med(RM): -6.82
MAD(RM): 13.65
med(sigma_phi): 0.49
sigma(RM)_obs: 30.33
nbeams: 26.15
Mean xray: 11.003 c/s/deg2


In [23]:
print("South Lobe:")
get_statistics(south_rm, south_sigma, rm_hdu.header)
print("Mean xray: {0:.3f} c/s/deg2".format(np.nanmean(south_xray)))

South Lobe:
<RM> : 10.15
sigma(RM): 46.72
med(RM): 6.82
MAD(RM): 27.30
med(sigma_phi): 0.74
sigma(RM)_obs: 46.72
nbeams: 23.37
Mean xray: 10.986 c/s/deg2
